In [1]:
import tensorflow as tf

import learned_simulator
import noise_utils
import reading_utils

import collections
import functools
import json
import os
import pickle
import tree

import numpy as np

import matplotlib.pyplot as plt


2021-10-12 17:34:20.315064: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:



!mkdir -p '/tmp/datasets'
!wget -O /tmp/datasets/metadata.json https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/WaterDrop/metadata.json
!wget -O /tmp/datasets/train.tfrecord https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/WaterDrop/train.tfrecord
!wget -O /tmp/datasets/valid.tfrecord https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/WaterDrop/valid.tfrecord
!wget -O /tmp/datasets/test.tfrecord https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/WaterDrop/test.tfrecord


--2021-10-12 17:07:14--  https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/WaterDrop/metadata.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.200.16, 142.250.200.48, 172.217.16.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.200.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 361 [application/octet-stream]
Saving to: ‘/tmp/datasets/metadata.json’

/tmp/datasets/metad 100%[===================>]     361  --.-KB/s    in 0s      

2021-10-12 17:07:14 (216 MB/s) - ‘/tmp/datasets/metadata.json’ saved [361/361]

--2021-10-12 17:07:15--  https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/WaterDrop/train.tfrecord
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.200.16, 142.250.200.48, 172.217.16.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.200.16|:443... connected.
HTTP request sent, awaiting response... 

In [2]:

INPUT_SEQUENCE_LENGTH = 6  # So we can calculate the last 5 velocities.
NUM_PARTICLE_TYPES = 9
KINEMATIC_PARTICLE_ID = 3

batch_size=2

def _read_metadata(data_path):
    with open(os.path.join(data_path, 'metadata.json'), 'rt') as fp:
        return json.loads(fp.read())

def prepare_inputs(tensor_dict):
  """Prepares a single stack of inputs by calculating inputs and targets.

  Computes n_particles_per_example, which is a tensor that contains information
  about how to partition the axis - i.e. which nodes belong to which graph.

  Adds a batch axis to `n_particles_per_example` and `step_context` so they can
  later be batched using `batch_concat`. This batch will be the same as if the
  elements had been batched via stacking.

  Note that all other tensors have a variable size particle axis,
  and in this case they will simply be concatenated along that
  axis.



  Args:
    tensor_dict: A dict of tensors containing positions, and step context (
    if available).

  Returns:
    A tuple of input features and target positions.

  """
  # Position is encoded as [sequence_length, num_particles, dim] but the model
  # expects [num_particles, sequence_length, dim].
  pos = tensor_dict['position']
  pos = tf.transpose(pos, perm=[1, 0, 2])

  # The target position is the final step of the stack of positions.
  target_position = pos[:, -1]

  # Remove the target from the input.
  tensor_dict['position'] = pos[:, :-1]

  # Compute the number of particles per example.
  num_particles = tf.shape(pos)[0]
  # Add an extra dimension for stacking via concat.
  tensor_dict['n_particles_per_example'] = num_particles[tf.newaxis]

  if 'step_context' in tensor_dict:
    # Take the input global context. We have a stack of global contexts,
    # and we take the penultimate since the final is the target.
    tensor_dict['step_context'] = tensor_dict['step_context'][-2]
    # Add an extra dimension for stacking via concat.
    tensor_dict['step_context'] = tensor_dict['step_context'][tf.newaxis]
  return tensor_dict, target_position

def batch_concat(dataset, batch_size):
  """We implement batching as concatenating on the leading axis."""

  # We create a dataset of datasets of length batch_size.
  windowed_ds = dataset.window(batch_size)

  # The plan is then to reduce every nested dataset by concatenating. We can
  # do this using tf.data.Dataset.reduce. This requires an initial state, and
  # then incrementally reduces by running through the dataset

  # Get initial state. In this case this will be empty tensors of the
  # correct shape.
  initial_state = tree.map_structure(
      lambda spec: tf.zeros(  # pylint: disable=g-long-lambda
          shape=[0] + spec.shape.as_list()[1:], dtype=spec.dtype),
      dataset.element_spec)

  # We run through the nest and concatenate each entry with the previous state.
  def reduce_window(initial_state, ds):
    return ds.reduce(initial_state, lambda x, y: tf.concat([x, y], axis=0))

  return windowed_ds.map(
      lambda *x: tree.map_structure(reduce_window, initial_state, x))


In [3]:

data_path = '/tmp/datasets'

# Loads the metadata of the dataset.
metadata = _read_metadata(data_path)
#Create a tf.data.Dataset from the TFRecord.

ds = tf.data.TFRecordDataset([os.path.join(data_path, 'train.tfrecord')])
ds = ds.map(functools.partial(reading_utils.parse_serialized_simulation_example, metadata=metadata))
# Splits an entire trajectory into chunks of 7 steps.
# Previous 5 velocities, current velocity and target.
split_with_window = functools.partial(
    reading_utils.split_trajectory,
    window_length=INPUT_SEQUENCE_LENGTH + 1)
ds = ds.flat_map(split_with_window)
# Splits a chunk into input steps and target steps
ds = ds.map(prepare_inputs)
# If in train mode, repeat dataset forever and shuffle.
ds = ds.repeat()
ds = ds.shuffle(512)
# Custom batching on the leading axis.
ds = batch_concat(ds, batch_size)


2021-10-12 17:34:25.924826: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-12 17:34:25.927156: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-12 17:34:25.978632: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-12 17:34:25.979165: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro P1000 computeCapability: 6.1
coreClock: 1.4805GHz coreCount: 5 deviceMemorySize: 3.92GiB deviceMemoryBandwidth: 74.65GiB/s
2021-10-12 17:34:25.979200: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-10-12 17:34:25.982095: I tensorflow/stream_executor/platform/default/dso_lo

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpla9lxp2j.py, line 13)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpla9lxp2j.py, line 13)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [4]:
for databatch in ds.as_numpy_iterator():
    print(databatch[0])
    break

2021-10-12 17:34:28.593657: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-12 17:34:28.613138: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3299990000 Hz


{'particle_type': array([5, 5, 5, ..., 5, 5, 5]), 'position': array([[[0.7813939 , 0.1687931 ],
        [0.7813903 , 0.16892076],
        [0.7813452 , 0.16894704],
        [0.7813512 , 0.16882114],
        [0.7814686 , 0.16851303],
        [0.78166634, 0.16801745]],

       [[0.7599475 , 0.16612233],
        [0.75968677, 0.1659991 ],
        [0.75944555, 0.1657685 ],
        [0.7593131 , 0.16538875],
        [0.75928825, 0.16482893],
        [0.7592968 , 0.16407833]],

       [[0.760049  , 0.16113883],
        [0.7598054 , 0.16108072],
        [0.7595709 , 0.16093822],
        [0.75943136, 0.16062841],
        [0.7593891 , 0.16010247],
        [0.7593813 , 0.15936947]],

       ...,

       [[0.13055547, 0.22743459],
        [0.13101469, 0.22861966],
        [0.13148275, 0.22974853],
        [0.13195878, 0.23081811],
        [0.13243766, 0.23182389],
        [0.13291474, 0.23276502]],

       [[0.2745001 , 0.12609279],
        [0.27160683, 0.1259225 ],
        [0.26871043, 0.12575364],

2021-10-12 17:34:29.323316: W tensorflow/core/framework/dataset.cc:477] Input of WindowDataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


In [5]:
noise_std=6.7e-4
latent_size=128
hidden_size=128
hidden_layers=2
message_passing_steps=10

In [6]:
"""Gets one step model for training simulation."""
metadata = _read_metadata(data_path)
model_kwargs = dict(
      latent_size=latent_size,
      mlp_hidden_size=hidden_size,
      mlp_num_hidden_layers=hidden_layers,
      num_message_passing_steps=message_passing_steps)
def _combine_std(std_x, std_y):
  return np.sqrt(std_x**2 + std_y**2)

Stats = collections.namedtuple('Stats', ['mean', 'std'])
vel_noise_std=noise_std
acc_noise_std=noise_std
"""Instantiates the simulator."""
# Cast statistics to numpy so they are arrays when entering the model.
cast = lambda v: np.array(v, dtype=np.float32)
acceleration_stats = Stats(cast(metadata['acc_mean']), _combine_std(cast(metadata['acc_std']), acc_noise_std))
velocity_stats = Stats(cast(metadata['vel_mean']),_combine_std(cast(metadata['vel_std']), vel_noise_std))
normalization_stats = {'acceleration': acceleration_stats, 'velocity': velocity_stats}



if 'context_mean' in metadata:
    context_stats = Stats(cast(metadata['context_mean']), cast(metadata['context_std']))
    normalization_stats['context'] = context_stats


simulator = learned_simulator.LearnedSimulator(
      num_dimensions=metadata['dim'],
      connectivity_radius=metadata['default_connectivity_radius'],
      graph_network_kwargs=model_kwargs,
      boundaries=metadata['bounds'],
      num_particle_types=NUM_PARTICLE_TYPES,
      normalization_stats=normalization_stats,
      particle_type_embedding_size=16)


#   def estimator_fn(features, labels, mode):
#     target_next_position = labels
#     # Sample the noise to add to the inputs to the model during training.
#     sampled_noise = noise_utils.get_random_walk_noise_for_position_sequence(
#         features['position'], noise_std_last_step=noise_std)
#     non_kinematic_mask = tf.logical_not(
#         get_kinematic_mask(features['particle_type']))
#     noise_mask = tf.cast(
#         non_kinematic_mask, sampled_noise.dtype)[:, tf.newaxis, tf.newaxis]
#     sampled_noise *= noise_mask

#     # Get the predictions and target accelerations.
#     pred_target = simulator.get_predicted_and_target_normalized_accelerations(
#         next_position=target_next_position,
#         position_sequence=features['position'],
#         position_sequence_noise=sampled_noise,
#         n_particles_per_example=features['n_particles_per_example'],
#         particle_types=features['particle_type'],
#         global_context=features.get('step_context'))
#     pred_acceleration, target_acceleration = pred_target

#     # Calculate the loss and mask out loss on kinematic particles/
#     loss = (pred_acceleration - target_acceleration)**2

#     num_non_kinematic = tf.reduce_sum(
#         tf.cast(non_kinematic_mask, tf.float32))
#     loss = tf.where(non_kinematic_mask, loss, tf.zeros_like(loss))
#     loss = tf.reduce_sum(loss) / tf.reduce_sum(num_non_kinematic)

In [7]:
initial_positions = databatch[0]['position'][:, 0:INPUT_SEQUENCE_LENGTH]
n_particles_per_example=databatch[0]['n_particles_per_example'],
particle_types=databatch[0]['particle_type']
global_context_step = None

In [8]:
simulator(initial_positions,
        n_particles_per_example=n_particles_per_example,
        particle_types=particle_types,
        global_context=global_context_step)

AttributeError: 'tuple' object has no attribute 'get_shape'